QISHA_PlotROICounts.ipynb
Created by JFranco, 10 SEP 2023

This is the fifth module in the QISHA pipepline. It is meant for plotting the results of a single experiment (i.e., not intended for batch analysis). The code will read in all puncta counts from a single ROI directory and plot the results. 

In [21]:
#                           *** LIBRARIES ***
import datetime
import numpy as np
import os
import matplotlib.image
import matplotlib.pyplot as plt
from cellpose import models , plot
import imageio
import pandas as pd
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [3]:
#                   *** GET TIME OF ANALYSIS START ***
# could turn this into a module?
toa = str(datetime.datetime.today()).split()                                   #Time of analysis
today = toa[0]
now = toa[1]
timestamp = today.replace('-','')+'-'+now.replace(':','')[:6]

In [14]:
#                   *** WHERE TO GET/SAVE DATA ***   
# Directories 
#   UPDATE THIS INFO TO REFLECT YOUR SYSTEM
dirMain = "/Users/joyfranco/Dropbox (Partners HealthCare)/JF_Shared/Data/CYS/CYS_003/"
dirPCs = dirMain+"ROIPuncta/PunctaCounts/"
dirMD = dirMain+"Metadata/"

#   ones that need to be made
dirAna = dirMain+"Analysis/"
dirPlts = dirAna+"Plots"+timestamp+"/"

In [15]:
# Load the metadata
fnMD = "CYS_003.Metadata.Samples.csv"
dfMD = pd.read_csv(dirMD+fnMD)

In [16]:
print(dfMD)

  Slides  Column      Row  AnimalID             Genotype Sex      DOB  \
0    All       1  1 and 2      1103  Ntng1Cre+/-;Ai14+/-   F   3/5/23   
1    All       2  1 and 2       689  Ntng1Cre+/-;Ai14+/-   M  2/28/22   
2    All       3  1 and 2       530  Ntng1Cre+/-;Ai14+/-   M  11/2/21   

   SacDate  AgeAtSac CollectionPrep         SampleID  
0  4/11/23        37     WSS_006.08  CYS_003.S11.TS2  
1   3/6/23       371      WSS_005.E  CYS_003.S11.TS4  
2   5/1/23       545     WSS_007.01  CYS_003.S10.TS5  


In [5]:
# Create directory for storing ROIs and Masks
if not os.path.exists(dirAna):
    os.mkdir(dirAna)
if not os.path.exists(dirPlts):
    os.mkdir(dirPlts)

In [6]:
# Get list of Puncta Counts to plot  
os.chdir(dirPCs)
pcs = os.listdir()

In [18]:
# Iterate through the list and generate a dataframe of compiled data

# Initialize dataframe for storing pcs
dfAll = pd.DataFrame()

for pc in pcs:
    # Gather the metadata for this sample
    fnBase = pc.split(".PunctaCounts",1)[0]
    # Index of the sample in the metadata sheet
    inSamp = dfMD.index[dfMD['SampleID']==fnBase].tolist()[0]
    # Get sample specific details
    animalID = dfMD.loc[inSamp]['AnimalID']
    geno = dfMD.loc[inSamp]['Genotype']
    sex = dfMD.loc[inSamp]['Sex']
    age = dfMD.loc[inSamp]['AgeAtSac']
    
    # Read in the puncta counts 
    dfPC = pd.read_csv(dirPCs+pc)

    # Add metadata
    dfPC["animalID"] = animalID
    dfPC["geno"] = geno
    dfPC["sex"] = sex
    dfPC["age"] = age

    # Append to main df
    dfAll = pd.concat([dfAll, dfPC])

In [20]:
print(dfAll)

          Count        ImageName  ROI_ID  animalID                 geno sex  \
0      1      8  CYS_003.S11.TS2       1      1103  Ntng1Cre+/-;Ai14+/-   F   
1      2      2  CYS_003.S11.TS2       2      1103  Ntng1Cre+/-;Ai14+/-   F   
2      3      6  CYS_003.S11.TS2       3      1103  Ntng1Cre+/-;Ai14+/-   F   
3      4      6  CYS_003.S11.TS2       4      1103  Ntng1Cre+/-;Ai14+/-   F   
4      5      4  CYS_003.S11.TS2       5      1103  Ntng1Cre+/-;Ai14+/-   F   
..   ...    ...              ...     ...       ...                  ...  ..   
155  156      2  CYS_003.S10.TS5     156       530  Ntng1Cre+/-;Ai14+/-   M   
156  157      1  CYS_003.S10.TS5     157       530  Ntng1Cre+/-;Ai14+/-   M   
157  158      6  CYS_003.S10.TS5     158       530  Ntng1Cre+/-;Ai14+/-   M   
158  159      3  CYS_003.S10.TS5     159       530  Ntng1Cre+/-;Ai14+/-   M   
159  160      1  CYS_003.S10.TS5     160       530  Ntng1Cre+/-;Ai14+/-   M   

     age  
0     37  
1     37  
2     37  
3     3

In [ ]:
####### GENERATE PLOTS #######
# Plot Response Maximum Fold Change
plt.figure(figsize=(4, 6))
sns.catplot(x='genotype',y='mxfc', hue=condition, 
            data=dfAll, kind="box",palette=colors,boxprops={'facecolor':'None'},linewidth=.5, 
            dodge=False, order=order, legend = False)
ax = sns.swarmplot(x='genotype', y='mxfc', hue=condition,
                   data=dfAll, palette=colors, 
                   dodge = False, size=4,  order=order, legend =False)
ax.set(xlabel='Genotype', ylabel='Maximum Fold Change from Baseline')
ax.set(ylim=(0, 0.6))
plt.xticks(rotation=45)
plt.savefig(dirTR+fnFigMaxFC_SB,  bbox_inches='tight', dpi = 300,
            format = figForm)